In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-midterm-p1/train.csv
/kaggle/input/2021-ml-midterm-p1/test.csv
/kaggle/input/2021-ml-midterm-p1/sample_submit.csv


In [2]:
# data load
train_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p1/train.csv')
test_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p1/test.csv')
submit = pd.read_csv('/kaggle/input/2021-ml-midterm-p1/sample_submit.csv')

print(train_data.info())
print(test_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trestbps  242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalach   242 non-null    int64  
 9   exang     242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slope     242 non-null    int64  
 12  ca        242 non-null    object 
 13  thal      242 non-null    object 
 14  target    242 non-null    int64  
dtypes: float64(1), int64(12), object(2)
memory usage: 28.5+ KB
None
   index  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      3   37    1   3       130   250    0        0      187      0      3.5   
1

In [3]:
# x, y, test data 처리
x_data = train_data.drop(['index', 'target'], axis=1)
y_data = train_data['target']
test_data = test_data.drop(['index'], axis=1)

In [4]:
# 데이터 정보 보기
print(train_data.info())
print(test_data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trestbps  242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalach   242 non-null    int64  
 9   exang     242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slope     242 non-null    int64  
 12  ca        242 non-null    object 
 13  thal      242 non-null    object 
 14  target    242 non-null    int64  
dtypes: float64(1), int64(12), object(2)
memory usage: 28.5+ KB
None
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   37    1   3       130   250    0        0      187      0      3.5      3   
1

In [5]:
# label encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_data['ca'] = le.fit_transform(x_data['ca'])
test_data['ca'] = le.transform(test_data['ca'])

x_data['thal'] = le.fit_transform(x_data['thal'])
test_data['thal'] = le.transform(test_data['thal'])

In [6]:
# split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.15, random_state=42)

In [7]:
# knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(weights = 'distance', n_neighbors = 8, p = 1)
knn.fit(x_train, y_train)
print(knn.score(x_val, y_val))

0.6486486486486487


In [8]:
# lgt
from sklearn.linear_model import LogisticRegression
lgt = LogisticRegression()
lgt.fit(x_train, y_train)
print(lgt.score(x_val, y_val))

0.5945945945945946


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [9]:
# lda
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)
print(lda.score(x_val, y_val))

0.5945945945945946


In [10]:
#submit
pred_knn = knn.predict(test_data)
submit['target'] = pred_knn
submit.to_csv('submit_knn.csv', header = True, index = False)

In [11]:
# 먼저 데이터를 로드 한 후 x, y, test 데이터로 나눠주고 index가 필요없다고 생각하여 index를 드랍하였다.
# 또한 object형이 너무많아서 object를 모두 실수화 해주었다.
# y데이터를 보니 범주형 데이터라 분류 문제라 판단하였고 분류 3개의 모델을 각각 학습해보고 예상 점수를  구해본 결과 knn이 가장 높게 나와 qda를선택했다.
# 또한 knn의 거리 가중치를 부여하였고 실험적으로 맨하탄 거리를 선택하였다.